In [1]:
from matplotlib import pyplot as plt

## Get video info

duration (sec) :  3560.045833
frame_count :  1708822

duration (sec) :  3560.083333
frame_count :  170884

In [8]:
"""get video info using ffmpeg.probe"""
import ffmpeg

video_path ='/local1/video-analysis/videos/706893_2024-05-28_15-15-38/ear/segment_000.mp4'
# video_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/figures/nose/ae/D=16_recon_ae_test.mp4'
# video_path = '/home/faeze.aminmansoor/video-analysis/videos/706893_2024-05-28_15-15-38/nose/nose_48/processed_video.mp4'

# print(hight, width, fps)
print('H : ', ffmpeg.probe(video_path)['streams'][0]['height'])
print('W : ', ffmpeg.probe(video_path)['streams'][0]['width'])
print('fps : ', ffmpeg.probe(video_path)['streams'][0]['r_frame_rate'])
print('duration (sec) : ', ffmpeg.probe(video_path)['streams'][0]['duration'])
print('frame_count : ', ffmpeg.probe(video_path)['streams'][0]['nb_frames'])
print('codec_name : ', ffmpeg.probe(video_path)['streams'][0]['codec_name'])
print('pixel_format : ', ffmpeg.probe(video_path)['streams'][0]['pix_fmt'])



ffmpeg.probe(video_path)

H :  105
W :  112
fps :  48/1
duration (sec) :  20.689583
frame_count :  994
codec_name :  h264
pixel_format :  yuv444p


{'streams': [{'index': 0,
   'codec_name': 'h264',
   'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10',
   'profile': 'High 4:4:4 Predictive',
   'codec_type': 'video',
   'codec_tag_string': 'avc1',
   'codec_tag': '0x31637661',
   'width': 112,
   'height': 105,
   'coded_width': 112,
   'coded_height': 105,
   'closed_captions': 0,
   'film_grain': 0,
   'has_b_frames': 2,
   'sample_aspect_ratio': '225:224',
   'display_aspect_ratio': '15:14',
   'pix_fmt': 'yuv444p',
   'level': 30,
   'chroma_location': 'left',
   'field_order': 'progressive',
   'refs': 1,
   'is_avc': 'true',
   'nal_length_size': '4',
   'id': '0x1',
   'r_frame_rate': '48/1',
   'avg_frame_rate': '477120/9931',
   'time_base': '1/15360',
   'start_pts': 630,
   'start_time': '0.041016',
   'duration_ts': 317792,
   'duration': '20.689583',
   'bit_rate': '80753',
   'bits_per_raw_sample': '8',
   'nb_frames': '994',
   'extradata_size': 50,
   'disposition': {'default': 1,
    'dub': 0,
    'ori

In [6]:
import ffmpeg

def get_video_info(video_path):
    try:
        probe = ffmpeg.probe(video_path)
        stream = probe['streams'][0]  # Extracting the first video stream

        print(f"H : {stream.get('height', 'N/A')}")
        print(f"W : {stream.get('width', 'N/A')}")
        print(f"FPS : {stream.get('r_frame_rate', 'N/A')}")  # FPS might be a fraction (e.g., '30000/1001')
        print(f"Duration (sec) : {stream.get('duration', 'N/A')}")
        print(f"Frame Count : {stream.get('nb_frames', 'N/A')}")
        print(f"Codec Name : {stream.get('codec_name', 'N/A')}")
        print(f"Pixel Format : {stream.get('pix_fmt', 'N/A')}")

    except ffmpeg.Error as e:
        print(f"Error retrieving video info: {e.stderr.decode()}")

# Define video path
video_path = '/local1/video-analysis/videos/706893_2024-05-28_15-15-38/ears/left_ear_.mp4'

# Run the function
get_video_info(video_path)

import fractions

fps_str = stream.get('r_frame_rate', '0/1')  # Default to '0/1' if missing
fps = float(fractions.Fraction(fps_str))  # Convert to float
print(f"FPS : {fps}")


H : 210
W : 225
FPS : 480/1
Duration (sec) : 3560.045833
Frame Count : 1708822
Codec Name : h264
Pixel Format : yuv444p


NameError: name 'stream' is not defined

## Trims to the first `t` seconds (if specified). Downsizes dimensions by a specified ratio. Adjusts the frame rate by a specified factor. Splits the video into segments of a specified duration.

In [2]:
import subprocess
import os

def process_video(input_video_path, output_directory, downsize_ratio=None, frame_rate_factor=None,
                   segment_duration=None, t_start=None, t_stop=None, frame_rate_mode="vfr"):
    """
    Processes a video with optional resizing, frame rate adjustment, trimming (using t_start and t_stop),
    and segmentation. Allows choosing frame rate mode (-vsync vfr or -vsync cfr).
    """
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Temporary file path for the processed video
    temp_video_path = os.path.join(output_directory, 'temp_video.mp4')

    # Build the FFmpeg command for processing
    ffmpeg_command = ['ffmpeg', '-i', input_video_path]

    # Apply trimming if t_start and t_stop are specified
    if t_start is not None:
        ffmpeg_command.extend(['-ss', str(t_start)])
    if t_stop is not None:
        ffmpeg_command.extend(['-to', str(t_stop)])

    # Get original frame rate if frame rate adjustment is needed
    original_frame_rate = None
    new_frame_rate = None
    N = None

    if frame_rate_factor is not None:
        probe = subprocess.run(
            ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
             '-show_entries', 'stream=r_frame_rate', '-of',
             'default=noprint_wrappers=1:nokey=1', input_video_path],
            capture_output=True, text=True, check=True
        )
        original_frame_rate = eval(probe.stdout.strip())  # Convert frame rate from string to float
        new_frame_rate = int(original_frame_rate * frame_rate_factor)
        N = int(1 / frame_rate_factor)  # Calculate frame step (e.g., N=10 for 480 → 48)

    # Construct filter list
    filter_list = []

    # Apply downscaling if requested
    if downsize_ratio is not None:
        filter_list.append(f"scale=iw*{downsize_ratio}:ih*{downsize_ratio}")

    # Apply frame rate reduction if needed
    if frame_rate_factor is not None and N is not None:
        if frame_rate_mode == "vfr":
            # Apply frame skipping using select filter
            filter_list.append(f"select='not(mod(n,{N}))'")
        elif frame_rate_mode == "cfr":
            # Enforce exact FPS by duplicating/removing frames if necessary
            filter_list.append(f"fps={new_frame_rate}")

    # Merge filters to avoid overwriting
    if filter_list:
        ffmpeg_command.extend(['-vf', ','.join(filter_list)])

    # Set vsync option **before** filters
    if frame_rate_factor is not None:
        if frame_rate_mode == "vfr":
            ffmpeg_command.insert(1, '-vsync')
            ffmpeg_command.insert(2, 'vfr')
        elif frame_rate_mode == "cfr":
            ffmpeg_command.insert(1, '-vsync')
            ffmpeg_command.insert(2, 'cfr')

    # Set the codec and output file
    ffmpeg_command.extend(['-c:v', 'libx264', '-crf', '18', '-preset', 'slow', temp_video_path])

    # Execute the FFmpeg command
    subprocess.run(ffmpeg_command, check=True)

    # Split the processed video into segments if 'segment_duration' is specified
    if segment_duration is not None:
        segment_output_path = os.path.join(output_directory, 'segment_%03d.mp4')
        split_command = [
            'ffmpeg', '-i', temp_video_path, '-c', 'copy',
            '-f', 'segment', '-segment_time', str(segment_duration), '-reset_timestamps', '1',
            segment_output_path
        ]
        subprocess.run(split_command, check=True)
        os.remove(temp_video_path)
    else:
        final_output_path = os.path.join(output_directory, 'processed_video.mp4')
        os.rename(temp_video_path, final_output_path)


In [4]:
bodypart = 'ear'

In [ ]:
input_video_path = f"/local1/video-analysis/videos/706893_2024-05-28_15-15-38/{bodypart}/left_ear_.mp4"
output_directory = f'/local1/video-analysis/videos/706893_2024-05-28_15-15-38/{bodypart}'

process_video(
    input_video_path,
    output_directory,
    downsize_ratio=None,        # Resize by 50% (properly applied)
    frame_rate_factor=None,     # Reduce to 10% of original frame rate (e.g., 480 → 48)
    segment_duration=None,       # Set to desired duration or None to skip
    t_start=2300, 
    t_stop=2500,                    # Trim first 30 sec
    frame_rate_mode="vfr"      # Choose "vfr" for dropping frames, "cfr" for forcing exact FPS
)

# def process_video(input_video_path, output_directory, downsize_ratio=None, frame_rate_factor=None,
#                    segment_duration=None, t_start=None, t_stop=None, frame_rate_mode="vfr")

ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-2)
  configuration: --prefix=/home/faeze.aminmansoor/miniconda3/envs/video-analysis-env --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1699729448698/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1699729448698/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1699729448698/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1699729448698/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-li

## Breakdown of command options

-c:v h264: This sets the video codec to H.264, a commonly used codec known for its efficient compression and high quality. H.264 is widely compatible with most video players and platforms.

-crf 0: The Constant Rate Factor (CRF) is a setting that adjusts the video quality. Setting -crf 0 results in lossless compression for H.264, meaning the video quality will be preserved exactly as in the input. (For lossy but efficient compression, a CRF value of 18-23 is typically used.)

-preset slow: This option defines the speed of the encoding process. Preset values (from "ultrafast" to "veryslow") impact both quality and file size; slower presets generally provide better compression. The slow preset means the encoding process will take more time but result in better compression efficiency (i.e., smaller file size for the same quality).

-c:a copy: This sets the audio codec, with copy indicating that the audio stream should be copied without re-encoding. This preserves the original audio quality and saves processing time, as the audio doesn’t need to be processed further.


## Get mata data of segment_000.mp4

In [5]:
# print(hight, width, fps)
import ffmpeg
video_path = output_directory + '/segment_000.mp4'
print('specs of the prosesseced video, first segment:')

print('H : ', ffmpeg.probe(video_path)['streams'][0]['height'])
print('W : ', ffmpeg.probe(video_path)['streams'][0]['width'])
print('fps : ', ffmpeg.probe(video_path)['streams'][0]['r_frame_rate'])
print('duration (sec) : ', ffmpeg.probe(video_path)['streams'][0]['duration'])
print('frame_count : ', ffmpeg.probe(video_path)['streams'][0]['nb_frames'])
print('codec_name : ', ffmpeg.probe(video_path)['streams'][0]['codec_name'])
print('pixel_format : ', ffmpeg.probe(video_path)['streams'][0]['pix_fmt'])

specs of the prosesseced video, first segment:
H :  124
W :  309
fps :  480/1
duration (sec) :  20.312500
frame_count :  9750
codec_name :  h264
pixel_format :  yuv444p


In [6]:
video_path

'/local1/video-analysis/videos/706893_2024-05-28_15-15-38/tongue_mouth/tongue_mouth-seg20sec-480Hz/segment_000.mp4'

In [ ]:
# import h5py

# # assume images are in an np array named "images_np"; this should be a an array of dtype
# # 'uint8', and values should be between 0 and 255. The data type is converted to float
# # and values are divided by 255 during the data loading process.

# Video data is assumed to be in a list, where each list element corresponds to a single trial, and is a numpy array of shape 
# (n_frames, n_channels, y_pix, x_pix).

# hdf5_file = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/VideoFolder/hdf5_file/data.hdf5'  # path needs to exist, but not 'data.hdf5' file

# with h5py.File(hdf5_file, 'w', libver='latest', swmr=True) as f:

#     # enable single write, multi-read - needed for simultaneous model fitting
#     f.swmr_mode = True

#     # create "image" HDF5 group
#     group_i = f.create_group('images')

#     # create a dataset for each trial within groups
#     for trial in range(len(images_np)):

#         # create dataset in "image" group
#         # images_np[trial] should be of shape (n_frames, n_channels, y_pix, x_pix)
#         group_i.create_dataset('trial_%04i' % trial, data=images_np[trial], dtype='uint8')



## make data.hdf5 file from segmented videos

In [23]:
"""Extracts frames from a given video file, adds a channel dimension for grayscale, and stores them in an HDF5 file, with dimensions 
(n_frames, 1, y_pix, x_pix)"""

import cv2
import os
import numpy as np
import h5py

# Video data is assumed to be in a list, where each list element corresponds to a single trial, and is a numpy array of shape 
# (n_frames, n_channels, y_pix, x_pix).

def extract_frames(video_path):
    """ Extracts frames from a given video file and adds a channel dimension for grayscale """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Add a channel dimension: (y_pix, x_pix) to (1, y_pix, x_pix)
        gray_frame = gray_frame[np.newaxis, :, :]  # Adds a new axis for the channel
        frames.append(gray_frame)
    cap.release()
    return np.array(frames, dtype=np.uint8)  # Shape becomes (n_frames, 1, y_pix, x_pix)

def videos_to_hdf5(video_folder, hdf5_path):
    """ Converts video segments into an HDF5 file with a specified shape per trial """
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]
    video_files.sort()  # Sort files to maintain order

    hdf5_file = hdf5_path  # Define the path to your HDF5 file
    # if hdf5_path dose not exist, create it
    if not os.path.exists(os.path.dirname(hdf5_file)):
        os.makedirs(os.path.dirname(hdf5_file))


    with h5py.File(hdf5_file, 'w', libver='latest', swmr=True) as f:
        f.swmr_mode = True
        group_i = f.create_group('images')
        for i, video_file in enumerate(video_files):
            video_path = os.path.join(video_folder, video_file)
            frames = extract_frames(video_path)
            # Store each video's frames in a separate dataset
            group_i.create_dataset(f'trial_{i:04d}', data=frames, dtype='uint8')

# Define paths and call the function
video_folder = '/local1/video-analysis/videos/706893_2024-05-28_15-15-38/ear/ear-seg20sec'  # Update this path to your video segments folder
hdf5_path = '/local1/video-analysis/videos/706893_2024-05-28_15-15-38/ear/hdf5-dir/data.hdf5'  # Update this path to where you want your HDF5 file
videos_to_hdf5(video_folder, hdf5_path)


/tmp/ipykernel_17641/2860369652.py:38: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(hdf5_file, 'w', libver='latest', swmr=True) as f:


In [19]:
"""get the size of data.hdf5 file"""

file_size = os.path.getsize(hdf5_path)  # Size in bytes
print(f"data.hdf5 file size: {file_size/8} Bytes")

data.hdf5 file size: 251205522.375 Bytes


### Get the number of frames in the selected_segments

In [20]:
"""get the dimensions of a dataset in an HDF5 file"""

import h5py

# Open the HDF5 file and access a specific dataset to print its dimensions
with h5py.File(hdf5_path, 'r') as file:
    data_shape = file['images/trial_0000'].shape  # Adjust the dataset path as needed
    print(f"Dimensions of the dataset for trial_0000  [(n_frames, n_channels, y_pix, x_pix)]: {data_shape}")

# print(hdf5_path)

Dimensions of the dataset for trial_0000  [(n_frames, n_channels, y_pix, x_pix)]: (1000, 1, 105, 112)


In [21]:
import h5py

def get_frame_counts(hdf5_path, selected_trials=None):
    """
    Reads an HDF5 file and extracts the number of frames for each selected trial.

    Parameters:
        hdf5_path (str): Path to the HDF5 file.
        selected_trials (list, optional): List of trial indices to include. If None, all trials will be used.

    Returns:
        dict: A dictionary with trial keys and their corresponding frame counts.
        int: The total number of frames across the selected trials.
        int: The total number of trials in the HDF5 file.
    """
    n_frames_dict = {}

    with h5py.File(hdf5_path, 'r') as f:
        images_group = f['images']
        total_num_trials = len(images_group.keys())  # Get the total number of trials

        # If no specific trials are selected, use all available trials
        if selected_trials is None:
            selected_trials = list(range(total_num_trials))

        # Loop over the selected trial indices
        for idx in selected_trials:
            trial_key = f"trial_{idx:04d}"
            if trial_key in images_group:
                n_frames = images_group[trial_key].shape[0]  # Extract number of frames
                n_frames_dict[trial_key] = n_frames

    # Compute total frames across selected trials
    total_num_frames = sum(n_frames_dict.values())

    return n_frames_dict, total_num_frames, total_num_trials



In [22]:
# Example usage
# hdf5_path = '/home/faeze.aminmansoor/video-analysis/scratch/706893_2024-05-28_15-15-38/data/xinxin/coupled-baiting_tongue_mouth/706893/2024-05-28/data.hdf5'

# Define the list of trial indices you are interested in (e.g., excluded segments)
# excluded_segments = [0, 1, 2, 3, 174, 175, 176, 177]
# selected_segments = excluded_segments
# print('Excluded segments:', excluded_segments)
# selected_segments = [0, 1, 2, 3]
selected_segments = list(range(178))  # Define specific trials to analyze

n_frames_dict, total_num_frames, total_num_frames_in_segments = get_frame_counts(hdf5_path, selected_segments)

# Print results
print("Total number of trials in the HDF5 file:", total_num_frames_in_segments)
print("Total number of frames for selected trials:", total_num_frames)
print("Frame counts per trial:", n_frames_dict)

Total number of trials in the HDF5 file: 178
Total number of frames for selected trials: 170883
Frame counts per trial: {'trial_0000': 1000, 'trial_0001': 1071, 'trial_0002': 1000, 'trial_0003': 1000, 'trial_0004': 889, 'trial_0005': 1000, 'trial_0006': 1000, 'trial_0007': 750, 'trial_0008': 1119, 'trial_0009': 978, 'trial_0010': 890, 'trial_0011': 1000, 'trial_0012': 1000, 'trial_0013': 973, 'trial_0014': 750, 'trial_0015': 997, 'trial_0016': 1000, 'trial_0017': 971, 'trial_0018': 1000, 'trial_0019': 986, 'trial_0020': 1000, 'trial_0021': 750, 'trial_0022': 1143, 'trial_0023': 870, 'trial_0024': 882, 'trial_0025': 979, 'trial_0026': 933, 'trial_0027': 1188, 'trial_0028': 968, 'trial_0029': 750, 'trial_0030': 1000, 'trial_0031': 975, 'trial_0032': 1000, 'trial_0033': 1000, 'trial_0034': 1000, 'trial_0035': 750, 'trial_0036': 1000, 'trial_0037': 972, 'trial_0038': 1000, 'trial_0039': 1000, 'trial_0040': 971, 'trial_0041': 1000, 'trial_0042': 1000, 'trial_0043': 750, 'trial_0044': 1000, 

### mkdir -p /local1/video-analysis/scratch/706893_2024-05-28_15-15-38/data/xinxin/coupled-baiting_tongue_mouth/706893/2024-05-28/
### mv hdf5_path /local1/video-analysis/scratch/706893_2024-05-28_15-15-38/data/xinxin/coupled-baiting_tongue_mouth/706893/2024-05-28/

## A few random functions

In [2]:
"""combine video segments into a single video file"""
import os
import subprocess

# Path to the directory containing the video segments
directory = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/'

# Create a text file to list the video segments from 351 to 361
file_list_path = os.path.join(directory, 'input_videos.txt')

with open(file_list_path, 'w') as file_list:
    for i in range(90, 101):  # 362 because range end is exclusive
        if i < 10:
            segment_path = os.path.join(directory, f'segment_00{i}.avi')
        elif i < 100:
            segment_path = os.path.join(directory, f'segment_0{i}.avi')
        else:
            segment_path = os.path.join(directory, f'segment_{i}.avi')
        # print(f"Processing: {segment_path}")
        if os.path.exists(segment_path):
            file_list.write(f"file '{segment_path}'\n")
        else:
            print(f"Warning: {segment_path} not found.")

# Output path for the combined video
output_video = os.path.join(directory, 'combined_video_90_to_100.avi')

# Run ffmpeg command to concatenate the videos
ffmpeg_command = [
    'ffmpeg',
    '-f', 'concat',
    '-safe', '0',
    '-i', file_list_path,
    '-c', 'copy',
    output_video
]

# Execute the ffmpeg command
subprocess.run(ffmpeg_command)

print(f"Video concatenation completed: {output_video}")


Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_090.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_091.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_092.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_093.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_094.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_095.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/face_50fr_10sec_clips/segment_096.avi
Processing: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_ra

ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

In [12]:
""" get an image and increase brightness  """
import ffmpeg

# Step 1: Convert HEIC to JPG
input_path = '/Users/faeze.aminmansoor/Downloads/IMG_5439.jpg'
# intermediate_path = '/Users/faeze.aminmansoor/Downloads/IMG_5439_intermediate.jpg'
output_path = '/Users/faeze.aminmansoor/Downloads/IMG_5439_brightened_02.jpg'

# Convert HEIC to intermediate JPG
# ffmpeg.input(input_path).output(intermediate_path).run()

# Step 2: Increase brightness on the JPG file
ffmpeg.input(input_path).output(output_path, vf='eq=brightness=0.2').run()


ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

(None, None)

In [23]:
"""read the input video and display a segment of it, from start_time to start_time + duration"""
import subprocess

# Define the input file path and parameters
# input_file = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/VideoFolder/face_paw_500fr_10sec_clips/segment_000.avi"
start_time = "10"  # Start time in seconds
duration = "10"    # Duration in seconds

# Run the FFmpeg command to display the video section
subprocess.run([
    "ffplay", 
    "-ss", start_time, 
    "-i", input_file, 
    "-t", duration
])


ffplay version 7.1 Copyright (c) 2003-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --

CompletedProcess(args=['ffplay', '-ss', '10', '-i', '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/side_camera.avi', '-t', '10'], returncode=0)

In [20]:
"""read the input video and save a segment of it, from start_time to start_time + duration """
output_file = "/Users/faeze.aminmansoor/Downloads/output_clip_.avi"
input_file = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/side_camera.avi'
# output_file = "output_clip_.avi"
subprocess.run([
    "ffmpeg", 
    "-ss", start_time, 
    "-i", input_file, 
    "-t", duration, 
    "-c", "copy", 
    output_file
])

ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

CompletedProcess(args=['ffmpeg', '-ss', '10', '-i', '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/684039_2023-12-13_10-46-46/side_camera.avi', '-t', '10', '-c', 'copy', '/Users/faeze.aminmansoor/Downloads/output_clip_.avi'], returncode=0)

## Display and save a selected frame from a video

In [1]:
"""This code snopeet shows how to extract a chosen frame from a video file and save it as an image file using OpenCV."""
import cv2

# Load the video
video_path = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/bottom_camera.avi'
cap = cv2.VideoCapture(video_path)

# Set to the desired frame (e.g., frame #100)
frame_number = 160
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

# Read the frame
ret, frame = cap.read()
if ret:
    # Save the frame as an image
    cv2.imwrite('frame.png', frame)
    print("Frame saved as 'frame.png'")
cap.release()


Frame saved as 'frame.png'


## Accept clicks on images, and stores the coordinates of the clicked point,

In [9]:
"""This code snippet shows how to display an image using Matplotlib and add a point to the plot when the user clicks on the image. 
The coordinates of the clicked points are stored in a list named "coords" and displayed after the plot window is closed."""
import matplotlib
matplotlib.use('TkAgg')  # Ensure an interactive backend is used
import matplotlib.pyplot as plt
import numpy as np

# Load the saved frame
img = plt.imread('frame.png')

# List to store the coordinates
coords = []

# Define the click event function
def onclick(event):
    if event.xdata is not None and event.ydata is not None:
        x, y = int(event.xdata), int(event.ydata)
        coords.append((x, y))
        print(f"Point added at ({x}, {y})")
        ax.plot(x, y, 'ro')  # Mark the point in red
        fig.canvas.draw_idle()  # Update the figure

# Create the figure and axes
fig, ax = plt.subplots()
ax.imshow(img)

# Connect the click event handler
cid = fig.canvas.mpl_connect('button_press_event', onclick)

# Display the plot
plt.show()

# After closing the plot window
fig.canvas.mpl_disconnect(cid)
print("Coordinates of clicked points:", coords)


Point added at (19, 346)
Point added at (217, 336)
Point added at (282, 375)
Point added at (337, 378)
Point added at (526, 531)
Point added at (11, 536)
Coordinates of clicked points: [(19, 346), (217, 336), (282, 375), (337, 378), (526, 531), (11, 536)]


In [10]:
coords

[(19, 346), (217, 336), (282, 375), (337, 378), (526, 531), (11, 536)]

## Masks the area outside a polygon defined by polygon_points on an image.

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def mask_outside_polygon(image_path, polygon_points):
    """
    Masks the area outside a polygon defined by polygon_points on an image.

    :param image_path: Path to the input image.
    :param polygon_points: List of (x, y) tuples defining the polygon.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Could not find or open the image at {image_path}")

    # Convert polygon points to a NumPy array
    points = np.array(polygon_points, dtype=np.int32)

    # Create a mask with the same size as the image, initialized to zero (black)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # Fill the polygon in the mask with white (value 255)
    cv2.fillPoly(mask, [points], 255)

    # Apply the mask to the image: keep the polygon and set the rest to black
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    # Display the original and masked image
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.title("Original Image")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    plt.subplot(1, 2, 2)
    plt.axis('off')
    plt.title("Masked Image")
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))

    plt.show()

# Example usage
image_path = './frame.png' # "your_image_path_here.jpg"  # Replace with the path to your image
polygon_points = coords #[(73, 5), (77, 166), (245, 162), (286, 137), (360, 127), (429, 52), (447, 5)]

try:
    mask_outside_polygon(image_path, polygon_points)
except FileNotFoundError as e:
    print(e)


## Crops a video to the smallest rectangle surrounding a polygon, masks the outside of the polygon black, and saves the output video.

In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def crop_and_mask_video(video_path, polygon_points, output_path):
    """
    Crops a video to the smallest rectangle surrounding a polygon, masks the outside of the polygon black, and saves the output video.

    :param video_path: Path to the input video.
    :param polygon_points: List of (x, y) tuples defining the polygon.
    :param output_path: Path to save the output video.
    """
    # Open the video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        raise FileNotFoundError(f"Could not find or open the video at {video_path}")

    # Convert polygon points to a NumPy array
    points = np.array(polygon_points, dtype=np.int32)

    # Find the bounding rectangle of the polygon
    x, y, w, h = cv2.boundingRect(points)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(video.get(cv2.CAP_PROP_FPS))
    output_size = (w, h)
    out = cv2.VideoWriter(output_path, fourcc, fps, output_size)

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Create a mask for the polygon
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [points], 255)

        # Apply the mask to the frame
        masked_frame = cv2.bitwise_and(frame, frame, mask=mask)

        # Crop the frame to the bounding rectangle
        cropped_frame = masked_frame[y:y+h, x:x+w]

        # Write the cropped frame to the output video
        out.write(cropped_frame)

    # Release resources
    video.release()
    out.release()

    print(f"Video saved to {output_path}")

# Example usage
# video_path = "your_video_path_here.avi"  # Replace with the path to your video
polygon_points = coords#[(73, 5), (77, 166), (245, 162), (286, 137), (360, 127), (429, 52), (447, 5)]
# output_path = "output_video.avi"
input_video_path = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/bottom_camera.avi'
output_video_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_left.mp4"

try:
    crop_and_mask_video(input_video_path, polygon_points, output_video_path)
except FileNotFoundError as e:
    print(e)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video saved to /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_left.mp4


## Stacks two videos vertically, with additional padding to ensure equal widths.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import ffmpeg

def collect_video_metadata(video_paths):
    """
    Collects metadata for a list of videos.

    :param video_paths: List of paths to video files.
    :return: List of metadata dictionaries.
    """
    metadata = []
    for video in video_paths:
        probe = ffmpeg.probe(video)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        video_meta = {
            "path": video,
            "width": video_stream['width'],
            "height": video_stream['height'],
            "frames": int(video_stream['nb_frames']),
            "frame_rate": eval(video_stream['r_frame_rate']),
            "duration": float(probe['format']['duration'])
        }
        metadata.append(video_meta)
    return metadata

def print_video_metadata(metadata):
    """
    Prints video metadata.

    :param metadata: List of metadata dictionaries.
    """
    print("Video Metadata:")
    for i, meta in enumerate(metadata):
        print(f"Video {i+1}:")
        print(f"  Path: {meta['path']}")
        print(f"  Width: {meta['width']} px")
        print(f"  Height: {meta['height']} px")
        print(f"  Frames: {meta['frames']}")
        print(f"  Frame Rate: {meta['frame_rate']} fps")
        print(f"  Duration: {meta['duration']} seconds\n")

def stack_videos_with_ffmpeg(video_paths, output_path, padding=10):
    """
    Stacks videos vertically with padding between them using FFmpeg.

    :param video_paths: List of paths to video files.
    :param output_path: Path to save the stacked video.
    :param padding: Padding size in pixels.
    """
    metadata = collect_video_metadata(video_paths)
    print_video_metadata(metadata)

    # Check for frame count and frame rate mismatches
    frames = {meta['frames'] for meta in metadata}
    frame_rates = {meta['frame_rate'] for meta in metadata}

    if len(frames) > 1 or len(frame_rates) > 1:
        print("Frame count or frame rate mismatch detected:")
        for i, meta in enumerate(metadata):
            print(f"Video {i+1}: {meta['frames']} frames, {meta['frame_rate']} fps, {meta['duration']} seconds")
        raise ValueError("All videos must have the same frame count and frame rate to stack.")

    # Prepare the filter_complex string
    filter_complex_parts = []
    max_width = max(meta['width'] for meta in metadata)
    for i, meta in enumerate(metadata):
        pad_filter = f"[{i}:v]pad=width={max_width}:height=ih+{padding if i < len(video_paths) - 1 else 0}:x={(max_width - meta['width']) // 2}:y=0:color=black[v{i}]"
        filter_complex_parts.append(pad_filter)
    stacked_filter = "".join(f"[v{i}]" for i in range(len(video_paths))) + f"vstack=inputs={len(video_paths)}[v]"
    filter_complex_parts.append(stacked_filter)

    # Build FFmpeg command
    ffmpeg_command = [
        "ffmpeg",
        *[item for video in video_paths for item in ("-i", video)],
        "-filter_complex", ";".join(filter_complex_parts),
        "-map", "[v]",
        "-c:v", "libx264",
        "-crf", "10",
        "-preset", "superfast",
        output_path
    ]

    # Run FFmpeg
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Video stacking completed successfully. Output saved at {output_path}.")
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg execution: {e}")

# Example usage
video1_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_right.mp4"  # Replace with the path to your first video
video2_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_left.mp4"  # Replace with the path to your second video
stacked_output_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/stacked_whiskers.mp4"

video_paths = [video1_path, video2_path]  # Replace with your video paths
stacked_output_path = stacked_output_path

try:
    stack_videos_with_ffmpeg(video_paths, stacked_output_path)
except FileNotFoundError as e:
    print(e)
except ValueError as e:
    print(e)

Video Metadata:
Video 1:
  Path: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_right.mp4
  Width: 374 px
  Height: 162 px
  Frames: 1708821
  Frame Rate: 480.0 fps
  Duration: 3560.04375 seconds

Video 2:
  Path: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_left.mp4
  Width: 516 px
  Height: 200 px
  Frames: 1708821
  Frame Rate: 480.0 fps
  Duration: 3560.04375 seconds



ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

Video stacking completed successfully. Output saved at /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/stacked_whiskers.mp4.


[out#0/mp4 @ 0x14c011b60] video:9415072kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.213981%
frame=1708821 fps=2478 q=-1.0 Lsize= 9435219kB time=00:59:20.03 bitrate=21711.4kbits/s speed=5.16x    
[libx264 @ 0x14c012cd0] frame I:6836  Avg QP: 3.09  size: 16312
[libx264 @ 0x14c012cd0] frame P:441984 Avg QP: 7.27  size:  9897
[libx264 @ 0x14c012cd0] frame B:1260001 Avg QP:11.15  size:  4091
[libx264 @ 0x14c012cd0] consecutive B-frames:  1.0%  1.5%  1.3% 96.2%
[libx264 @ 0x14c012cd0] mb I  I16..4: 56.1% 14.7% 29.2%
[libx264 @ 0x14c012cd0] mb P  I16..4:  4.8%  3.2%  4.1%  P16..4: 30.0%  0.0%  0.0%  0.0%  0.0%    skip:58.0%
[libx264 @ 0x14c012cd0] mb B  I16..4:  0.3%  0.2%  0.2%  B16..8: 22.9%  0.0%  0.0%  direct: 8.8%  skip:67.7%  L0:38.0% L1:39.1% BI:23.0%
[libx264 @ 0x14c012cd0] 8x8 transform intra:25.5% inter:15.3%
[libx264 @ 0x14c012cd0] coded y,uvDC,uvAC intra: 56.0% 27.6% 21.6% inter: 18.0% 3.6% 0.7%
[libx264 @ 0x14c012cd0] i16 v,h,dc,p: 73% 17%  9%

: 

In [ ]:
def stack_videos_vertically(video1_path, video2_path, output_path):
    """
    Stacks two videos vertically. If their widths don't match, adds padding to ensure equal widths.

    :param video1_path: Path to the first video.
    :param video2_path: Path to the second video.
    :param output_path: Path to save the stacked video.
    """
    # Open the video files
    video1 = cv2.VideoCapture(video1_path)
    video2 = cv2.VideoCapture(video2_path)

    if not video1.isOpened() or not video2.isOpened():
        raise FileNotFoundError("Could not open one or both video files.")

    # Get properties of the videos
    fps = int(video1.get(cv2.CAP_PROP_FPS))
    width1 = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width2 = int(video2.get(cv2.CAP_PROP_FRAME_WIDTH))
    height2 = int(video2.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate the new width (maximum of both widths)
    new_width = max(width1, width2)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_size = (new_width, height1 + height2)
    out = cv2.VideoWriter(output_path, fourcc, fps, output_size)

    while True:
        ret1, frame1 = video1.read()
        ret2, frame2 = video2.read()

        if not ret1 and not ret2:
            break

        # Add padding to make the widths equal
        if ret1:
            pad1 = new_width - width1
            frame1 = cv2.copyMakeBorder(frame1, 0, 0, 0, pad1, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        else:
            frame1 = np.zeros((height1, new_width, 3), dtype=np.uint8)

        if ret2:
            pad2 = new_width - width2
            frame2 = cv2.copyMakeBorder(frame2, 0, 0, 0, pad2, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        else:
            frame2 = np.zeros((height2, new_width, 3), dtype=np.uint8)

        # Stack the frames vertically
        stacked_frame = np.vstack((frame1, frame2))

        # Write the stacked frame to the output video
        out.write(stacked_frame)

    # Release resources
    video1.release()
    video2.release()
    out.release()

    print(f"Stacked video saved to {output_path}")

# Example usage for stacking videos
video1_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_right.mp4"  # Replace with the path to your first video
video2_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/whiskers_bottom_left.mp4"  # Replace with the path to your second video
stacked_output_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/whiskers/stacked_whiskers.mp4"

try:
    stack_videos_vertically(video1_path, video2_path, stacked_output_path)
except FileNotFoundError as e:
    print(e)

## Calculates the bounding box coordinates of a set of points

In [26]:
"""This code snippet calculates the bounding box coordinates of a set of points in a list. 
The minimum and maximum x and y coordinates are found, and the top-left and bottom-right corners of the bounding box are displayed."""
import numpy as np

# Example list of points (more can be added)
points = coords

# Convert the list of points to a numpy array for easier processing
points_array = np.array(points)

# Find the minimum and maximum x and y coordinates
min_x, min_y = points_array.min(axis=0) # Top-left corner
max_x, max_y = points_array.max(axis=0) # Bottom-right corner

# Calculate the bounding box
bounding_box = ((min_x, min_y), (max_x, max_y))

print("Bounding Box Coordinates:")
print(f"Top-left corner: ({min_x}, {min_y})")
print(f"Bottom-right corner: ({max_x}, {max_y})")


Bounding Box Coordinates:
Top-left corner: (267, 81)
Bottom-right corner: (515, 406)


## Mask the area outside the bounding box black

In [5]:
"""This code snippet demonstrates how to mask an image using a bounding box. The area outside the bounding box is set to black, 
and the masked image is displayed."""
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load the image
img = Image.open('frame.png')
img_np = np.array(img)  # Convert image to NumPy array for masking

# Define the bounding box coordinates (example coordinates)
# Replace these values with your actual bounding box coordinates
# x_min, y_min = 150, 200 # Top-left corner
# x_max, y_max = 717, 245 # Bottom-right corner
x_min, y_min = bounding_box[0] # Top-left corner
x_max, y_max = bounding_box[1] # Bottom-right corner

# Create a mask: set the area outside the bounding box to black
masked_img = img_np.copy()
masked_img[:y_min, :] = 0  # Top area
masked_img[y_max:, :] = 0  # Bottom area
masked_img[:, :x_min] = 0  # Left area
masked_img[:, x_max:] = 0  # Right area

# Display the masked image
plt.imshow(masked_img)
plt.axis('off')  # Hide axes for cleaner display
plt.show()

# Optionally, save the masked image
Image.fromarray(masked_img).save('masked_frame.png')

# print(x_min, y_min, x_max, y_max)
print(bounding_box)


NameError: name 'bounding_box' is not defined

In [1]:
bounding_box = ((267, 81), (515, 406))

### Some comments on high quality code for high-quality output

command = [

"ffmpeg", "-i", input_video_path,

"-vf", crop_filter,

"-c:v", "libx264",       # High-quality H.264 codec

"-preset", "veryslow",   # Best compression quality

"-crf", "18",            # High-quality constant rate factor

"-c:a", "aac",           # High-quality audio codec

"-b:a", "320k",          # High-quality audio bitrate

output_video_path

]

Set -crf to a low value (e.g., 18 or 0 for lossless).

Use -preset veryslow for better compression and quality.

Ensure -c:v libx264 or -c:v libx265 for efficient codecs.

Include high-quality audio with -c:a aac -b:a 320k.

#### Trade-Offs

File Size: Higher quality produces larger files.

Processing Time: Slower presets (veryslow) take significantly longer to encode.

## Crops a video to a specified bounding box and saves the cropped output. 


In [32]:
import subprocess
import os
import ffmpeg

def crop_video(input_video_path, output_video_path, bounding_box):
    """
    Crops a video to a specified bounding box and saves the output.
    
    Parameters:
    - input_video_path (str): Path to the input video.
    - output_video_path (str): Path to save the cropped video.
    - bounding_box (tuple): ((x1, y1), (x2, y2)) representing the top-left and bottom-right corners.
    """

    # Collect video metadata to validate compatibility
    probe = ffmpeg.probe(input_video_path)
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    video_meta = {
        "path": input_video_path,
        "width": video_stream['width'],
        "height": video_stream['height'],
        "frames": int(video_stream['nb_frames']),
        "frame_rate": eval(video_stream['r_frame_rate']),
        "duration": float(probe['format']['duration'])
    }

    # Print metadata
    print("Video Metadata:")
    print(f"  Path: {video_meta['path']}")
    print(f"  Width: {video_meta['width']} px")
    print(f"  Height: {video_meta['height']} px")
    print(f"  Frames: {video_meta['frames']}")
    print(f"  Frame Rate: {video_meta['frame_rate']} fps")
    print(f"  Duration: {video_meta['duration']} seconds\n")


    # Extract bounding box coordinates
    (x1, y1), (x2, y2) = bounding_box
    width = x2 - x1
    height = y2 - y1

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

    # FFmpeg crop filter: crop=w: h: x: y
    crop_filter = f"crop={width}:{height}:{x1}:{y1}"
    
    # FFmpeg command
    command = [
        "ffmpeg", "-i", input_video_path,
        "-vf", crop_filter,
        "-c:v", "libx264",  # Video codec
        "-preset", "ultrafast",  # Encoding speed. Options: ultrafast, superfast, veryfast, faster, fast, medium, slow, slower, veryslow. The -preset option controls encoding speed and compression efficiency. Slower presets result in better compression (smaller file size for the same quality). The default is medium.
        "-crf", "1",       # Quality (lower is better, 23 is default). Use a CRF value between 0 (lossless) and 18 (visually lossless). CRF controls the quality of the output video. f you want perfect quality and don't care about file size, set -crf 0
        "-an",              # Remove audio
        output_video_path
    ]

    # high quality code for high-quality output
    # command = [
    # "ffmpeg", "-i", input_video_path,
    # "-vf", crop_filter,
    # "-c:v", "libx264",       # High-quality H.264 codec
    # "-preset", "veryslow",   # Best compression quality
    # "-crf", "18",            # High-quality constant rate factor
    # "-c:a", "aac",           # High-quality audio codec
    # "-b:a", "320k",          # High-quality audio bitrate
    # output_video_path
    # ]

    # Set -crf to a low value (e.g., 18 or 0 for lossless).
    # Use -preset veryslow for better compression and quality.
    # Ensure -c:v libx264 or -c:v libx265 for efficient codecs.
    # Include high-quality audio with -c:a aac -b:a 320k.
    # Trade-Offs
    # File Size: Higher quality produces larger files.
    # Processing Time: Slower presets (veryslow) take significantly longer to encode.


    # Run the command
    subprocess.run(command, check=True)
    print(f"Cropped video saved to {output_video_path}")

# Example Usage
input_video_path = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/bottom_camera.avi'
output_video_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/tongue_bottom.mp4"
# bounding_box = ((282, 140), (381, 200))  # Example bounding box
bounding_box = bounding_box
crop_video(input_video_path, output_video_path, bounding_box)
      

Video Metadata:
  Path: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/bottom_camera.avi
  Width: 720 px
  Height: 540 px
  Frames: 1708821
  Frame Rate: 480.0 fps
  Duration: 3560.04375 seconds



ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

CalledProcessError: Command '['ffmpeg', '-i', '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/bottom_camera.avi', '-vf', 'crop=248:325:267:81', '-c:v', 'libx264', '-preset', 'ultrafast', '-crf', '1', '-an', '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/tongue_bottom.mp4']' returned non-zero exit status 228.

## Stacks multiple videos side by side and saves the output to a file.

In [ ]:
import subprocess

def rotate_video(input_path, output_path):
    """Rotates a video 90 degrees clockwise."""
    try:
        subprocess.run(
            ["ffmpeg", "-i", input_path, "-vf", "transpose=1", "-c:a", "copy", output_path],
            check=True
        )
        print(f"Rotated video saved at: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error rotating video: {e}")

# Example usage
input_path  = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/bottom_tongue.mp4 -vf transpose=1 -c:a copy output_video.mp4"
rotated_video_path = "/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/bottom_tongue_rotated.mp4"
rotate_video(input_path, rotated_video_path)


In [37]:
import subprocess
import ffmpeg

def stack_videos_with_padding(video_paths, output_path, padding=10):
    """
    Stacks videos side by side with padding, ensuring equal heights with additional black space.

    Args:
        video_paths (list): List of input video file paths.
        output_path (str): Path to save the stacked video.
        padding (int): Width of the padding (in pixels) between videos.
    """
    if len(video_paths) < 2:
        raise ValueError("You need at least two videos to stack them side by side.")

    # Collect video metadata to validate compatibility
    metadata = []
    for video in video_paths:
        probe = ffmpeg.probe(video)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        video_meta = {
            "path": video,
            "width": video_stream['width'],
            "height": video_stream['height'],
            "frames": int(video_stream['nb_frames']),
            "frame_rate": eval(video_stream['r_frame_rate']),
            "duration": float(probe['format']['duration'])
        }
        metadata.append(video_meta)

    # Print metadata
    print("Video Metadata:")
    for i, meta in enumerate(metadata):
        print(f"Video {i+1}:")
        print(f"  Path: {meta['path']}")
        print(f"  Width: {meta['width']} px")
        print(f"  Height: {meta['height']} px")
        print(f"  Frames: {meta['frames']}")
        print(f"  Frame Rate: {meta['frame_rate']} fps")
        print(f"  Duration: {meta['duration']} seconds\n")

    # Check for frame count and frame rate mismatches
    frames = {meta['frames'] for meta in metadata}
    frame_rates = {meta['frame_rate'] for meta in metadata}

    if len(frames) > 1 or len(frame_rates) > 1:
        print("Frame count or frame rate mismatch detected:")
        for i, meta in enumerate(metadata):
            print(f"Video {i+1}: {meta['frames']} frames, {meta['frame_rate']} fps, {meta['duration']} seconds")
        raise ValueError("All videos must have the same frame count and frame rate to stack.")

    # Prepare the filter_complex string
    filter_complex_parts = []
    max_height = max(meta['height'] for meta in metadata)
    for i, meta in enumerate(metadata):
        pad_filter = f"[{i}:v]pad=width=iw+{padding if i < len(video_paths) - 1 else 0}:height={max_height}:x=0:y={(max_height - meta['height']) // 2}:color=black[v{i}]"
        filter_complex_parts.append(pad_filter)
    stacked_filter = "".join(f"[v{i}]" for i in range(len(video_paths))) + f"hstack=inputs={len(video_paths)}[v]"
    filter_complex_parts.append(stacked_filter)

    # Build FFmpeg command
    ffmpeg_command = [
        "ffmpeg",
        *[item for video in video_paths for item in ("-i", video)],
        "-filter_complex", ";".join(filter_complex_parts),
        "-map", "[v]",
        "-c:v", "libx264",
        "-crf", "10",
        "-preset", "slow",
        output_path
    ]

    # Run FFmpeg
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Video stacking completed successfully. Output saved at {output_path}.")
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg execution: {e}")

# Example usage
video_paths = [
    '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/right_tongue.mp4',
    '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/bottom_tongue.mp4',
    '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/tongue/left_tongue.mp4'
]
output_path = '/Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/eyes/stacked_eye_output_with_padding.mp4'

stack_videos_with_padding(video_paths, output_path, padding=7)


Video Metadata:
Video 1:
  Path: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/eyes/right_eye_slower.mp4
  Width: 104 px
  Height: 76 px
  Frames: 1708822
  Frame Rate: 480.0 fps
  Duration: 3560.045833 seconds

Video 2:
  Path: /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/eyes/left_eye.mp4
  Width: 99 px
  Height: 60 px
  Frames: 1708822
  Frame Rate: 480.0 fps
  Duration: 3560.045833 seconds



ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1699729534217/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

Video stacking completed successfully. Output saved at /Users/faeze.aminmansoor/Documents/Allen/Xinxin_raw_data/706893_2024-05-28_15-15-38/eyes/stacked_eye_output_with_padding.mp4.


[out#0/mp4 @ 0x136692b30] video:1552630kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.062278%
frame=1708822 fps= 28 q=-1.0 Lsize= 1569124kB time=00:59:20.03 bitrate=3610.7kbits/s speed=0.0578x    
[libx264 @ 0x136693c90] frame I:6836  Avg QP:12.03  size:  8647
[libx264 @ 0x136693c90] frame P:867249 Avg QP:14.95  size:  1629
[libx264 @ 0x136693c90] frame B:834737 Avg QP:21.54  size:   141
[libx264 @ 0x136693c90] consecutive B-frames: 30.0% 10.3% 13.0% 46.7%
[libx264 @ 0x136693c90] mb I  I16..4:  0.6% 15.8% 83.6%
[libx264 @ 0x136693c90] mb P  I16..4:  0.0%  0.1%  0.8%  P16..4: 54.5% 15.8% 15.4%  0.0%  0.0%    skip:13.5%
[libx264 @ 0x136693c90] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 28.4%  4.6%  2.0%  direct: 5.4%  skip:59.5%  L0:38.8% L1:50.1% BI:11.1%
[libx264 @ 0x136693c90] 8x8 transform intra:11.0% inter:9.8%
[libx264 @ 0x136693c90] direct mvs  spatial:100.0% temporal:0.0%
[libx264 @ 0x136693c90] coded y,u,v intra: 69.5% 50.9% 57.4% inter: 31.0% 5.

## This is an alternative code for click detection

In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

def on_click(event):
    if event.xdata is not None and event.ydata is not None:
        print(f"Clicked at ({event.xdata}, {event.ydata})")

fig, ax = plt.subplots()
fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()


In [18]:
%matplotlib notebook
%matplotlib tk

In [9]:
import cv2

coords = []

# Define the click event
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        coords.append((x, y))
        print(f"Point added at ({x}, {y})")
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
        cv2.imshow('image', img)

# Load the image
img = cv2.imread('frame.png')

# Display the image and set the mouse callback
cv2.imshow('image', img)
cv2.setMouseCallback('image', click_event)

# Wait until a key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Coordinates of clicked points:", coords)


KeyboardInterrupt: 

# Extract frame timestamps of original and downsized video using frame_metadata.json metadata file


### In Brief: 

ffprobe -i /home/faeze.aminmansoor/video-analysis/videos/706893_2024-05-28_15-15-38/nose/stacked_nose.mp4 -show_frames -select_streams v:0 -print_format json > /home/faeze.aminmansoor/video-analysis/videos/706893_2024-05-28_15-15-38/nose/frame_meta_data.json

head -n 50 /home/faeze.aminmansoor/video-analysis/videos/706893_2024-05-28_15-15-38/nose/frame_meta_data.json

In [ ]:
import json
import pandas as pd

# frame_meta_data_path = '/home/faeze.aminmansoor/video-analysis/videos/706893_2024-05-28_15-15-38/nose/frame_meta_data.json'

# Load JSON file
with open("frame_metadata.json", "r") as file:
    data = json.load(file)

# Extract frame information
frames = data["frames"]
timestamps = []

for frame in frames:
    timestamps.append({
        "Frame_ID": frame["coded_picture_number"],
        "Timestamp(s)": float(frame["pts_time"]),  # Convert to float for calculations
        "Duration": float(frame.get("pkt_duration_time", 0))  # Handle missing values
    })

# Convert to DataFrame
df_48 = pd.DataFrame(timestamps)

# Adjust the 'Timestamp(s)' column to start from 0
df_48["Timestamp(s)"] -= df_48["Timestamp(s)"].iloc[0]

# Save to CSV
df_48.to_csv("frame_timestamps.csv", index=False)

# Display first few rows
print(df_48.head(10))
print(df_48.tail(10))

df_48.to_csv('df_48.csv', index=False)



### ffprobe -i segment_000.mp4 -show_frames -select_streams v:0 -print_format json > frame_metadata.json

our MP4 file contains video metadata, and from this ffprobe output, we can extract useful details

This should have saved per-frame metadata in frame_metadata.json.

### head -n 50 frame_metadata.json

Your frame_metadata.json file contains per-frame timestamps extracted by FFmpeg. Here are the relevant fields:

Field	                        Description
"pts_time" (pkt_pts_time)	    Frame presentation timestamp in seconds
"pkt_dts_time" (pkt_dts)	    Decoding timestamp (useful for reordering)
"pkt_duration_time"	            Frame duration in seconds
"coded_picture_number"	        Frame index (zero-based)


If the file (frame_metadata.json) is empty or doesn't contain pkt_pts_time, try this modified command:
### ffprobe -i segment_000.mp4 -show_entries frame=pkt_pts_time -of json > frame_timestamps.json

### Extract Frame Timestamps to CSV
To extract and save the frame timestamps into a structured format, use this Python script:

import json
import pandas as pd

# Load JSON file
with open("frame_metadata.json", "r") as file:
    data = json.load(file)

# Extract frame timestamps
frames = data["frames"]
timestamps = []

for frame in frames:
    timestamps.append({
        "Frame_ID": frame["coded_picture_number"],
        "Timestamp": float(frame["pts_time"]),  # Convert to float for calculations
        "Duration": float(frame.get("pkt_duration_time", 0))  # Handle missing values
    })

# Convert to DataFrame
df = pd.DataFrame(timestamps)

# Save to CSV
df.to_csv("frame_timestamps.csv", index=False)

# Display first few rows
print(df.head())

### Expected Output (frame_timestamps.csv)
Frame_ID	Timestamp (s)	Duration (s)
0	0.003971	0.002083
1	0.006055	0.002083
2	0.008139	0.002083
3	0.010223	0.002083


1. Presentation Timestamp (PTS):

The PTS indicates the exact moment a frame should be displayed during playback. It's crucial for synchronizing audio and video streams, ensuring that each frame appears at the intended time. For example, a PTS of 5.0 seconds means the frame should be shown precisely at the 5-second mark during playback. 
EN.WIKIPEDIA.ORG

2. Decoding Timestamp (DTS):

The DTS specifies when a frame should be decoded. This is particularly important for frames that rely on data from other frames, such as B-frames, which reference both previous and subsequent frames. Proper decoding order ensures that frames are ready for presentation at their designated PTS. 
ELECARD.COM

3. Frame Duration:

Frame duration denotes the length of time a frame is displayed. It's typically the inverse of the frame rate. For instance, at a frame rate of 25 frames per second (fps), each frame has a duration of 0.04 seconds (1/25). This metric ensures smooth motion by maintaining consistent timing between frames. 
STACKOVERFLOW.COM

4. Frame Type:

Frames in compressed video are categorized based on their compression method:

I-Frames (Intra-coded): Self-contained frames that don't reference other frames.

P-Frames (Predicted): Frames that reference preceding frames to save space.

B-Frames (Bidirectional): Frames that reference both preceding and following frames for higher compression efficiency.

Understanding frame types is essential for efficient video compression and decompression processes. 
EN.WIKIPEDIA.ORG

5. Frame Size:

Frame size refers to the amount of data a frame occupies, typically measured in bytes. Factors influencing frame size include resolution, compression level, and frame complexity. Higher resolution and more detailed frames generally result in larger frame sizes.

These metadata elements are integral to video encoding and playback, ensuring that frames are decoded and presented in the correct sequence and at the appropriate times.